In [1]:
import os
os.chdir('../')
os.getcwd()
%load_ext autoreload
%autoreload 2
import src.forecaster.diagnostic as diagnostic
import  src.forecaster.modelil as mod
import src.forecaster.utilitaires as util
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


raw_master = pd.read_csv('data/raw/raw_master_il_1019.csv')

fc_eln = pd.read_csv('data/raw/DI/di_forecast.csv')

In [2]:
mod = mod.Modelil(raw_master)

max_date_available = mod.all_sales.calendar_yearmonth.max()
filter_date = min(201908, max_date_available)
dwps = util.create_list_period(201601, filter_date, False)
dwp, dtp = util.get_all_combination_date(dwps, 10)

print("creating the main table")
table_all_features = mod.create_all_features(dwp, dtp)

creating the main table


In [3]:
from copy import deepcopy

XGBOOST_PARAMETERS = {
            'max_depth': 11,  # 25
            'gamma': 0.02,
            'subsample': 0.4,
            'n_estimators': 27,
            'learning_rate': 0.1,
            'n_jobs': 12,
            'verbosity': 2
        }
dico_params_eib = {}
for i in range(1,11):
    dico_params_eib[i] = XGBOOST_PARAMETERS.copy()

#dico_params[7]['n_estimators'] = 49
dico_params_eib[5]['n_estimators'] = 27 
dico_params_eib[6]['n_estimators'] = 45
dico_params_eib[7]['n_estimators'] = 31  
dico_params_eib[8]['n_estimators'] = 47 
dico_params_eib[9]['n_estimators'] = 13 
dico_params_eib[10]['n_estimators'] = 27

dico_params_di = deepcopy(dico_params_eib)
dico_params_di[6]['n_estimators'] = 81

In [4]:
params={}
params['eib']= dico_params_eib
params['di'] = dico_params_di

In [5]:
dwp_test = util.create_list_period(201702, 201902, False)

In [6]:
res = mod.recreate_past_forecasts(table_all_features, dwp_test, params, max_available_date=201908)

predicted=27407.108326, expected=26773.784319
predicted=31380.995714, expected=24897.214795
predicted=33643.380152, expected=23976.806097
predicted=31913.692143, expected=23070.055628
predicted=32037.792502, expected=21263.557260
predicted=31056.047304, expected=20267.222169
predicted=31417.487570, expected=19468.268955
predicted=28177.750548, expected=19472.006769
predicted=26123.957984, expected=18793.847278
predicted=28020.544990, expected=18349.638576
predicted=25575.036511, expected=18007.993867
predicted=23696.513233, expected=17767.760054
predicted=19959.817757, expected=16807.961734
predicted=20167.674789, expected=16325.512226
predicted=19958.365633, expected=16157.630615
predicted=16348.163342, expected=15514.130931
predicted=16043.535836, expected=15127.874365
predicted=16228.510283, expected=16185.722152
predicted=17704.285254, expected=16498.640706
predicted=16487.291223, expected=15868.570867
predicted=15947.374483, expected=13054.733503
predicted=15925.580400, expected=1

In [7]:
test = diagnostic.Diagnostic(cvr=res[(res.horizon==6)], raw_master=raw_master, postprocess='indep')
temp = test.run_test(plot=False, prediction_horizon=6)


*** Year 2018 ***
        il:	Target=     57462	Pred=     59438	Bias=   -0.0333	FA=    0.7136	FA'=    0.7038
       eib:	Target=     38034	Pred=     34145	Bias=    0.1202	FA=     0.625	FA'=    0.6634
        di:	Target=     19427	Pred=     24557	Bias=   -0.2089	FA=    0.5794	FA'=    0.4683

*** H1 2019 (until August) ***
        il:	Target=     34249	Pred=     35749	Bias=    -0.042	FA=    0.7754	FA'=    0.7655
       eib:	Target=     20421	Pred=     21361	Bias=   -0.0413	FA=    0.6752	FA'=    0.6603
        di:	Target=     13827	Pred=     15409	Bias=   -0.1026	FA=    0.6821	FA'=    0.6458


# KPIS with atleast 2 years of data at date when predciting

In [8]:
month_min=24 # minimal age in month at date when predicting
pred_horizon = 6
test = diagnostic.Diagnostic(cvr=res[(res.horizon==6)], raw_master=raw_master, postprocess='indep')
temp = test.run_test(plot=False, prediction_horizon=6, MINIMUM_HISTO = month_min + pred_horizon)


*** Year 2018 ***
        il:	Target=     55266	Pred=     57668	Bias=   -0.0417	FA=    0.7167	FA'=    0.7044
       eib:	Target=     36082	Pred=     32682	Bias=    0.1106	FA=    0.6311	FA'=    0.6659
        di:	Target=         0	Pred=         0	Bias=       nan	FA=       nan	FA'=       nan

*** H1 2019 (until August) ***
        il:	Target=     33920	Pred=     35411	Bias=   -0.0421	FA=    0.7758	FA'=    0.7659
       eib:	Target=     20092	Pred=     21005	Bias=   -0.0407	FA=    0.6736	FA'=    0.6587
        di:	Target=      4217	Pred=      4593	Bias=    -0.082	FA=     0.804	FA'=    0.7865


In [9]:
import numpy as np

fc_eln['cycle_month'] = pd.to_datetime(fc_eln['cycle_month'], format= "%Y-%m")
fc_eln['forecasted_month'] = pd.to_datetime(fc_eln['forecasted_month'], format= "%Y-%m")

fc_eln['horizon']  = 12 * (fc_eln['forecasted_month'].dt.year - fc_eln['cycle_month'].dt.year) + (fc_eln['forecasted_month'].dt.month - fc_eln['cycle_month'].dt.month)

fc_eln['horizon'] = fc_eln['horizon'].astype(int)

fc_eln['sku_wo_pkg'] = fc_eln.sku_w_pkg.str.split('_').str[0:3].str.join('_') + '_'+ fc_eln.sku_w_pkg.str.split('_').str[4]

fc_eln.horizon += 2
fc_eln['label'] = 'di'
fc_eln.rename(columns={'forecasted_month':'date_to_predict'}, inplace=True)
fc_eln.rename(columns={'cycle_month':'date_when_predicting'}, inplace=True)
fc_eln.rename(columns={'forecast':'prediction'}, inplace=True)
fc_eln['date_when_predicting'] = fc_eln['date_when_predicting'].dt.strftime('%Y%m')

fc_eln_di = fc_eln.groupby(['date_to_predict', 'sku_wo_pkg', 'horizon'])['prediction'].sum().reset_index()

tempdi = fc_eln_di.copy()
tempeib = res[res.label=='eib'].copy()

tempdi['date_to_predict'] = pd.to_datetime(tempdi['date_to_predict']).dt.year.astype(
            str) + pd.to_datetime(tempdi['date_to_predict']).dt.month.astype(str).str.zfill(2)
tempdi['date_to_predict'] = tempdi['date_to_predict'].astype(int)
tempdi['label'] = 'di'

In [10]:
tempdi.rename(columns={'prediction': 'preddi'}, inplace=True)
tempeib.rename(columns={'prediction': 'predeib'}, inplace=True)
tempil = pd.merge(
tempeib[['date_to_predict', 'sku_wo_pkg', 'predeib', 'horizon']],
tempdi[['date_to_predict', 'sku_wo_pkg', 'preddi', 'horizon']],
how='left',
on=['date_to_predict', 'sku_wo_pkg', 'horizon']
)
tempdiaug = tempil.copy()
tempdiaug['preddi'] = tempil['preddi'].fillna(0)
tempdiaug['label']='di'
tempdi = tempdiaug
tempil['prediction'] = tempil['preddi'].fillna(0) + tempil['predeib'].fillna(0)

tempil['label'] = 'il'
tempdi.rename(columns={'preddi': 'prediction'}, inplace=True)
tempeib.rename(columns={'predeib': 'prediction'}, inplace=True)
rescombined= pd.concat([tempdi, tempeib, tempil], sort=True).drop(['preddi', 'predeib'], axis=1, errors="ignore")

# KPIs with DI preds replaced by ELN predicitons 

In [11]:
test = diagnostic.Diagnostic(cvr=rescombined[(rescombined.horizon==6)], raw_master=raw_master, postprocess='indep')
temp = test.run_test(plot=False, prediction_horizon=6)


*** Year 2018 ***
        il:	Target=     57462	Pred=     57415	Bias=    0.0008	FA=    0.7428	FA'=    0.7431
       eib:	Target=     38034	Pred=     34145	Bias=    0.1202	FA=     0.625	FA'=    0.6634
        di:	Target=     19427	Pred=     23270	Bias=   -0.1652	FA=    0.6194	FA'=    0.5441

*** H1 2019 (until August) ***
        il:	Target=     34249	Pred=     36937	Bias=   -0.0728	FA=    0.7966	FA'=    0.7807
       eib:	Target=     20421	Pred=     21361	Bias=   -0.0413	FA=    0.6752	FA'=    0.6603
        di:	Target=     13827	Pred=     15575	Bias=   -0.1122	FA=    0.7833	FA'=    0.7559


# BELOW KPIS MIXED AF AND ELN

In [12]:
from src.forecaster.utilitaires import fa_score, fa_score_prime, bias_score, add_sku_age
master_il = add_sku_age(raw_master, 'date', ['sku_wo_pkg'],
                                         ['offtake_di', 'offtake_il', 'offtake_eib'],
                                         Thre_abs=12, Thre_rel=0.02)



cvr = test.ensure_output_format(res[(res.horizon==6)], True)

age = master_il[['date', 'sku_wo_pkg', 'country', 'brand', 'tier', 'stage', 'age_offtake_di',
                              'age_offtake_il', 'age_offtake_eib']][master_il.date >= cvr.date.min()]
age = age.rename(columns={'date': 'date'})
age['date'] = pd.to_datetime(age['date'], format='%Y%m')

cvr = cvr.merge(age, on=['date', 'sku_wo_pkg'], how='left', validate='m:1').fillna(-99)

In [13]:
res_corrected = res.copy()

In [14]:
cvr['date_to_predict'] = pd.to_datetime(cvr['date'], format='%Y-%m-%d')
cvr['date_to_predict'] = cvr['date_to_predict'].dt.strftime('%Y%m').astype(int)
res_corrected = pd.merge(res_corrected, cvr[['sku_wo_pkg', 'date_to_predict', 'age_offtake_di', 'horizon']], on = ['date_to_predict', 'horizon', 'sku_wo_pkg'] , how = 'left')

In [15]:
tmp=res_corrected[(res_corrected['label']=='di')&(res_corrected.age_offtake_di>=30)]
tmp.rename(columns={'prediction' : 'pred2'},inplace=True)
tmp.date_to_predict = tmp.date_to_predict.astype(int)
final_res = pd.merge(rescombined, tmp[['sku_wo_pkg', 'date_to_predict', 'label', 'horizon', 'pred2']], how = 'left', on = ['sku_wo_pkg', 'date_to_predict', 'label', 'horizon'])

In [17]:
final_res.loc[final_res.pred2>0, 'prediction'] = final_res.loc[final_res.pred2>0, 'pred2']

In [18]:
test = diagnostic.Diagnostic(cvr=final_res[(final_res.horizon==6)], raw_master=raw_master, postprocess='indep')
temp = test.run_test(plot=False, prediction_horizon=6)


*** Year 2018 ***
        il:	Target=     57462	Pred=     57415	Bias=    0.0008	FA=    0.7428	FA'=    0.7431
       eib:	Target=     38034	Pred=     34145	Bias=    0.1202	FA=     0.625	FA'=    0.6634
        di:	Target=     19427	Pred=     23270	Bias=   -0.1652	FA=    0.6194	FA'=    0.5441

*** H1 2019 (until August) ***
        il:	Target=     34249	Pred=     36937	Bias=   -0.0728	FA=    0.7966	FA'=    0.7807
       eib:	Target=     20421	Pred=     21361	Bias=   -0.0413	FA=    0.6752	FA'=    0.6603
        di:	Target=     13827	Pred=     15359	Bias=   -0.0997	FA=    0.7772	FA'=    0.7526


In [20]:
tempdi = final_res[final_res.label == 'di']
tempeib = final_res[final_res.label == 'eib']
tempil = final_res[final_res.label == 'il']

tempdi.rename(columns={'prediction': 'preddi'}, inplace=True)
tempeib.rename(columns={'prediction': 'predeib'}, inplace=True)
tempil = pd.merge(
    tempdi[['date_to_predict', 'sku_wo_pkg', 'preddi', 'horizon']],
    tempeib[['date_to_predict', 'sku_wo_pkg', 'predeib', 'horizon']],
    how='left',
    on=['date_to_predict', 'sku_wo_pkg', 'horizon']
)
tempil['prediction'] = tempil['preddi'] + tempil['predeib']

tempil['label'] = 'il'
tempdi.rename(columns={'preddi': 'prediction'}, inplace=True)
tempeib.rename(columns={'predeib': 'prediction'}, inplace=True)

finafinal=pd.concat([tempdi, tempeib, tempil], sort=True).drop(['preddi', 'predeib'], axis=1, errors="ignore")

In [21]:
test = diagnostic.Diagnostic(cvr=finafinal[(finafinal.horizon==6)], raw_master=raw_master, postprocess='indep')
temp = test.run_test(plot=False, prediction_horizon=6)


*** Year 2018 ***
        il:	Target=     57462	Pred=     57415	Bias=    0.0008	FA=    0.7428	FA'=    0.7431
       eib:	Target=     38034	Pred=     34145	Bias=    0.1202	FA=     0.625	FA'=    0.6634
        di:	Target=     19427	Pred=     23270	Bias=   -0.1652	FA=    0.6194	FA'=    0.5441

*** H1 2019 (until August) ***
        il:	Target=     34249	Pred=     36720	Bias=   -0.0673	FA=    0.7936	FA'=    0.7787
       eib:	Target=     20421	Pred=     21361	Bias=   -0.0413	FA=    0.6752	FA'=    0.6603
        di:	Target=     13827	Pred=     15359	Bias=   -0.0997	FA=    0.7772	FA'=    0.7526
